<a href="https://colab.research.google.com/github/estudos-wanderson/Modelagem-Matem-tica/blob/main/Tema_3_Sistemas_de_Equa%C3%A7%C3%B5es_Lineares_e_Ajuste_de_Curvas_em_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Método de eliminação de Gauss**

### **Processo de Eliminação de Gauss**

Considere um sistema de equações lineares $AX=B$ com $n$ equações e $n$ incógnitas. O objetivo é transformar a matriz de coeficientes $A$ em uma matriz triangular superior $U$ zerando todos os elementos abaixo da diagonal principal.

1. Passo 1 (para a coluna 1)
    - Escolha o primeiro elemento da primeira linha, $a_{11}$, como pivô.
    - Se $a_{11}=0$, troque a primeira linha com uma linha abaixo onde o elemento correspondente na coluna 1 seja não nulo. Se todos os elementos da primeira coluna forem zero, o processo de eliminação para esta coluna não é possível, então prossiga para a próxima coluna.
    - Para cada linha $i$ abaixo da primeira (para $i=2, \dots, n$), calcule o multiplicador $m_{i1} = \frac{a_{i1}}{a_{11}}$.
    - Substitua a linha $L_i$ por $L_i - m_{i1}L_1$. Este processo zera o elemento $a_{i1}$.
    - Repita o processo para todas as linhas abaixo da primeira.

2. Passo k (para a coluna k)
    - Ignore as $k-1$ primeiras linhas e colunas.
    - Considere o elemento $a_{kk}$ da matriz transformada como o novo pivô.
    - Se $a_{kk}=0$, troque a linha $k$ com uma linha abaixo que tenha um elemento não nulo na coluna $k$.
    - Para cada linha $i$ abaixo da linha $k$ (para $i=k+1, \dots, n$), calcule o multiplicador $m_{ik} = \frac{a_{ik}}{a_{kk}}$.
    - Substitua a linha $L_i$ por $L_i - m_{ik}L_k$.
    - Repita este processo até que a matriz $A$ se torne uma matriz triangular superior $U$.

O sistema original $AX=B$ é transformado no sistema triangular superior $UX=C$, onde $C$ é a coluna dos termos independentes $B$ transformada. O sistema final terá a forma:

$$
\begin{pmatrix}
u_{11} & u_{12} & \cdots & u_{1n} \\
0 & u_{22} & \cdots & u_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & u_{nn}
\end{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{pmatrix}
=
\begin{pmatrix}
c_1 \\
c_2 \\
\vdots \\
c_n
\end{pmatrix}
$$

3. Retrossubstituição

Após a eliminação de Gauss, o sistema de equações lineares é transformado em um sistema triangular superior $UX=C$, que é mais simples de resolver. O processo de retrossubstituição permite encontrar os valores das incógnitas a partir da última equação, "subindo" e substituindo os valores já encontrados.

- *Passo 1*: Resolva a última equação para $x_n$.
$$x_n = \frac{c_n}{u_{nn}}$$
- *Passo 2*: Substitua o valor de $x_n$ na penúltima equação e resolva para $x_{n-1}$.
- *Passo 3*: Continue "subindo" e substituindo os valores já encontrados nas equações anteriores, até resolver para $x_1$.

A fórmula geral para a retrossubstituição é:

Para $i = n-1, n-2, \dots, 1$:
$$x_i = \frac{1}{u_{ii}} \left( c_i - \sum_{j=i+1}^{n} u_{ij}x_j \right)$$

In [10]:
import numpy as np

def change_rows(A, i, j):
    A[[i, j]] = A[[j, i]]

def search_pivot_row(A, k):
    for i in range(k+1, len(A)):
        if A[i, k] != 0:
            return i
    return None

def gauss_elimination(A: np.array, B: np.array):
    n = len(B)
    if A.shape[0] != n or A.shape[1] != n:
        raise ValueError("A and B must have the same number of rows")

    if A.dtype != np.float64:
        A = A.astype(np.float64)
        B = B.astype(np.float64)

    for k in range(n - 1):
        if A[k, k] == 0:
            pivot_row = search_pivot_row(A, k)
            if pivot_row is None:
                raise ValueError("No unique solution exists")
            change_rows(A, k, pivot_row)
            change_rows(B, k, pivot_row)

        for i in range(k + 1, n):
            m = A[i, k] / A[k, k]
            A[i, k:] = A[i, k:] - m * A[k, k:]
            B[i] = B[i] - m * B[k]

    if A[n - 1, n - 1] == 0:
        raise ValueError("No unique solution exists")

    x = np.zeros(n)
    x[n - 1] = B[n - 1] / A[n - 1, n - 1]
    for i in range(n - 2, -1, -1):
        sum = 0
        for j in range(i + 1, n):
            sum += A[i, j] * x[j]
        x[i] = 1 / A[i, i] * (B[i] - sum)

    return x

Exemplo

In [11]:
A = np.array([[1, 1, 1], [2,1,1], [1,2,3]])
B = np.array([4,6,7])

sol = gauss_elimination(A, B)
print(sol)

[2. 1. 1.]


### **Processo de Eliminação de Gauss-Jordan**

O método de Eliminação de Gauss-Jordan é uma variação do método de Gauss que transforma a matriz de coeficientes $A$ em uma **matriz diagonal identidade** $I$, em vez de uma matriz triangular superior $U$. Isso elimina a necessidade da retrossubstituição, pois a solução do sistema de equações é obtida diretamente.

1.  **Passo 1 (para a coluna 1)**
    * Escolha o primeiro elemento da primeira linha, $a_{11}$, como pivô.
    * Se $a_{11}=0$, troque a primeira linha com uma linha abaixo onde o elemento correspondente na coluna 1 seja não nulo. Se todos os elementos da primeira coluna forem zero, o processo de eliminação não é possível, então prossiga para a próxima coluna.
    * Para normalizar o pivô, divida a primeira linha por $a_{11}$, tornando o novo pivô igual a 1.
    * Para cada linha $i$ (para $i=2, \dots, n$), calcule o multiplicador $m_{i1} = a_{i1}$.
    * Substitua a linha $L_i$ por $L_i - m_{i1}L_1$. Este processo zera todos os elementos abaixo do pivô na coluna 1.

2.  **Passo k (para a coluna k)**
    * Considere o elemento $a_{kk}$ da matriz transformada como o novo pivô.
    * Se $a_{kk}=0$, troque a linha $k$ com uma linha abaixo que tenha um elemento não nulo na coluna $k$.
    * Para normalizar o pivô, divida a linha $k$ por $a_{kk}$, tornando o novo pivô igual a 1.
    * Para cada linha $i$ (para $i=1, \dots, n$ e $i \ne k$), calcule o multiplicador $m_{ik} = a_{ik}$.
    * Substitua a linha $L_i$ por $L_i - m_{ik}L_k$. Este processo zera todos os elementos acima e abaixo do pivô na coluna $k$.
    * Repita este processo até que a matriz $A$ se torne a matriz identidade $I$.

O sistema original $AX=B$ é transformado no sistema diagonal $IX=D$, onde $D$ é a coluna dos termos independentes $B$ transformada. O sistema final terá a forma:

$$
\begin{pmatrix}
1 & 0 & \cdots & 0 \\
0 & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 1
\end{pmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
\vdots \\
x_n
\end{pmatrix}
=
\begin{pmatrix}
d_1 \\
d_2 \\
\vdots \\
d_n
\end{pmatrix}
$$

A solução é obtida diretamente, pois $x_i = d_i$ para $i=1, 2, \dots, n$.

In [8]:
import numpy as np

def change_rows(A, i, j):
    A[[i, j]] = A[[j, i]]

def search_pivot_row(A, k):
    for i in range(k + 1, len(A)):
        if A[i, k] != 0:
            return i
    return None

def gauss_jordan_elimination(A: np.array, B: np.array):
    n = len(B)
    if A.shape[0] != n or A.shape[1] != n:
        raise ValueError("A and B must have the same number of rows")

    if A.dtype != np.float64:
        A = A.astype(np.float64)
        B = B.astype(np.float64)

    for k in range(n):
        # Handle zero pivot by searching for a non-zero row
        if A[k, k] == 0:
            pivot_row = search_pivot_row(A, k)
            if pivot_row is None:
                raise ValueError("No unique solution exists")
            change_rows(A, k, pivot_row)
            change_rows(B, k, pivot_row)

        # Normalize the pivot row
        pivot_val = A[k, k]
        A[k] = A[k] / pivot_val
        B[k] = B[k] / pivot_val

        # Eliminate elements in all other rows
        for i in range(n):
            if i != k:
                multiplier = A[i, k]
                A[i] = A[i] - multiplier * A[k]
                B[i] = B[i] - multiplier * B[k]

    # The solution is the transformed B vector
    x = B

    return x

Exemplo

In [12]:
A = np.array([[1,  1, 1], [2, 1, 1], [1, 2, 3]])
B = np.array([4, 6, 7])

sol = gauss_jordan_elimination(A, B)
print(sol)

[2. 1. 1.]


# **Decomposição LU**

Considere o sistema $Ax=b$. Podemos reescrevê-lo como $LUx=b$. Para começar, definimos $Ux=y$. Com isso, o sistema se torna $Ly=b$. Como $L$ é triangular inferior, resolver $Ly=b$ para encontrar $y$ é fácil usando substituição direta.

Com o vetor $y$ definido, resolvemos $Ux=y$ para encontrar $x$. Como $U$ é triangular superior, a solução é simples por meio de substituição regressiva. Além disso, o determinante de $A$ é o produto dos determinantes de $L$ e $U$. Se $L$ tiver 1s na diagonal principal, $\det(L)=1$ e $\det(A)=\det(U)$, que é o produto dos elementos da diagonal de $U$.

## **Condições para a decomposição LU**

Para que uma matriz $A$ admita uma decomposição $LU$ única, com a condição de que a matriz $L$ tenha todos os elementos da sua diagonal principal iguais a 1, é necessário que todos os menores principais de $A$ sejam diferentes de zero. O que é um menor principal?

Se $A$ é uma matriz $n \times n$:

- $A_1$ é a submatriz $1 \times 1$ no canto superior esquerdo de $A$.
- $A_2$ é a submatriz $2 \times 2$ no canto superior esquerdo de $A$.

    $\vdots$

- $A_k$ é a submatriz $k \times k$ no canto superior esquerdo de $A$.


Com isso, se $\det(A_k) \neq 0$ para $k=1,2,\dots,n-1$, existe uma única matriz triangular inferior $L$ com $l_{ii}=1$ e uma única matriz triangular superior $U$, tal que:

$$A=LU$$

# *Fórmula Geral para o Cálculo de L e U*

Para uma matriz $A$ de ordem $n \times n$, os elementos $u_{ij}$ de $U$ e $l_{ij}$ de $L$ são calculados da seguinte forma:

## 1. Para os elementos de U

Para $i \leq j$:
$$u_{ij} = a_{ij} - \sum_{k=1}^{i-1} l_{ik} \cdot u_{kj}$$

Para a primeira linha de $U$ ($i=1$), o somatório é vazio, então:
$$u_{1j} = a_{1j} \quad \text{para} \quad j=1, \dots, n$$

## 2. Para os elementos de L

Para $i > j$, com $l_{jj}=1$:
$$l_{ij} = \frac{1}{u_{jj}} \left( a_{ij} - \sum_{k=1}^{j-1} l_{ik} \cdot u_{kj} \right)$$

Para a primeira coluna de $L$ ($j=1$), o somatório é vazio, então:
$$l_{i1} = \frac{1}{u_{11}} (a_{i1}) \quad \text{para} \quad i=2, \dots, n$$

In [18]:
import numpy as np

def lu_decomposition(A: np.array):
    n = len(A)
    if A.shape[0] != n or A.shape[1] != n:
        raise ValueError("A must be a square matrix")

    if A.dtype != np.float64:
        A = A.astype(np.float64)

    L = np.identity(n, dtype=np.float64)
    U = np.zeros([n, n], dtype=np.float64)

    for i in range(n):
        # Calculate U[i, j]
        for j in range(i, n):
            sum_val = 0.0
            for k in range(i):
                sum_val += L[i, k] * U[k, j]
            U[i, j] = A[i, j] - sum_val

        # Calculate L[j, i]
        for j in range(i + 1, n):
            sum_val = 0.0
            for k in range(i):
                sum_val += L[j, k] * U[k, i]
            L[j, i] = (A[j, i] - sum_val) / U[i, i]

    return L, U

In [20]:
def solve_lu(A: np.array, B: np.array):
    L, U = lu_decomposition(A)
    y = np.linalg.solve(L, B)
    x = np.linalg.solve(U, y)
    return x

In [22]:
A = np.array([[1, 2, 3], [4, 7, 6], [5, 8, 6]])
B = np.array([1, 2, 3])

sol = solve_lu(A, B)
print(sol)

[ 3.         -2.          0.66666667]
